In [12]:
import numpy as np
import tomlkit
import pandas as pd
import yaml
from IPython.display import display
import importlib
import copy

import study_lib

In [9]:
base_config_yaml = """
candidates: 5
voters: 555
considerations:
- Likability:
    mean: 0.25
- Irrational:
    sigma: 1.0
    camps: 7
    individualism_deg: 30
- Issues:
    issues:
    - sigma: 1.0
      halfcsep: 1.5
      halfvsep: 2.0
    - sigma: 0.5
      halfcsep: 1.0
      halfvsep: 1.0
    - sigma: 0.5
      halfcsep: 2.0
      halfvsep: 2.0
methods:
- Plurality:
    strat: Honest
- Plurality:
    strat: Strategic
- Range:
    nranks: 10
    strat: Honest
- Range:
    nranks: 10
    strat: Strategic
    strategic_stretch_factor: 1.5
- Range:
    nranks: 2
    strat: Honest
- Range:
    nranks: 2
    strat: Strategic
    strategic_stretch_factor: 4.0
- InstantRunoff: {}
- STAR:
    strat: Honest
- STAR:
    strat: Strategic
    strategic_stretch_factor: 1.5
"""
config = yaml.safe_load(base_config_yaml)

In [6]:
importlib.reload(study_lib)
do_run = study_lib.do_run 
get_fields = study_lib.get_fields
run_experiment = study_lib.run_experiment
cs = study_lib.config_series

In [10]:
df = run_experiment(cs(config, 'candidates', [2, 3, 4, 5, 6]), trials=20000)
df

,candidates,pl_h_mR,pl_h_pi,pl_s_mR,pl_s_pi,range_10_h_mR,range_10_h_pi,range_10_s_mR,range_10_s_pi,aprv_h_mR,aprv_h_pi,aprv_s_mR,aprv_s_pi,IRV_h_mR,IRV_h_pi,star_6_h_mR,star_6_h_pi,star_6_s_mR,star_6_s_pi
0,2,0.098800,95.060,0.098800,95.060,0.098800,95.06,0.098800,95.060,0.098800,95.060,0.098800,95.060,0.098800,95.060,0.098800,95.060,0.098800,95.060
1,3,0.270134,75.620,0.061412,90.555,0.028775,92.97,0.026235,93.670,0.030271,93.045,0.033494,93.150,0.062691,90.495,0.031671,93.480,0.033707,93.095
2,4,0.271646,69.060,0.067976,86.420,0.018384,92.21,0.016475,92.985,0.023352,91.055,0.022504,92.050,0.054008,87.915,0.019985,92.460,0.022196,92.000
3,5,0.291097,62.620,0.089046,81.005,0.014469,91.59,0.012359,92.580,0.020815,89.535,0.018996,90.715,0.053992,85.075,0.015777,91.675,0.019103,90.700
4,6,0.288888,58.845,0.109776,76.830,0.012475,91.63,0.010945,92.170,0.021126,88.390,0.016927,90.370,0.053940,83.325,0.014438,91.115,0.016636,90.280


In [19]:
config_like_only = copy.deepcopy(config)
config_like_only['candidates'] = 4
config_like_only['considerations'] = yaml.safe_load(
'''
- Likability:
    mean: 1.0
'''
)
result = do_run(config_like_only, 5)
display(result['cand_regret'][0])
display(result['cov_matrix'][0])

<Array [0, 1.16, 1.71, 1.13] type='4 * ?float64'>

<Array [[0], [0, 0], [...], [0, 0, 0, 0]] type='4 * option[var * ?float64]'>

With likability as the only consideration, all voter sets of utilities are identical to each other. Variances and covariances are zero.

In [20]:
def get_corr_coeffs(result, trial_no):
    covm = result['cov_matrix'][trial_no]
    ncand = len(covm)
    cc = np.eye(ncand, dtype=np.float64)
    sigmas = []
    for i in range(ncand):
        sigmas.append(np.sqrt(covm[(i, i)]))
    # print(sigmas)
    for ix in range(ncand):
        for iy in range(ix):
            cc[(ix, iy)] = covm[(ix, iy)] / sigmas[ix] / sigmas[iy]
    return cc

In [21]:
display(get_corr_coeffs(result, 0))

/tmp/ipykernel_202370/3163430169.py:11: RuntimeWarning: invalid value encountered in scalar divide
  cc[(ix, iy)] = covm[(ix, iy)] / sigmas[ix] / sigmas[iy]


array([[ 1.,  0.,  0.,  0.],
       [nan,  1.,  0.,  0.],
       [nan, nan,  1.,  0.],
       [nan, nan, nan,  1.]])

In [25]:
config_irr = copy.deepcopy(config)
config_irr['candidates'] = 4
config_irr['voters'] = 100001
config_irr['considerations'] = yaml.safe_load(
'''
- Irrational:
    sigma: 1.0
    camps: 2
    individualism_deg: 90
'''
)
result = do_run(config_irr, 5)
display(get_corr_coeffs(result, 0))
display(get_corr_coeffs(result, 1))


array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 4.42752037e-04,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.29308904e-03,  1.46733920e-03,  1.00000000e+00,
         0.00000000e+00],
       [ 5.90097806e-04,  3.55551757e-03, -1.01498929e-03,
         1.00000000e+00]])

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.0032342 ,  1.        ,  0.        ,  0.        ],
       [ 0.00343857, -0.00119389,  1.        ,  0.        ],
       [ 0.0019555 ,  0.00102327,  0.00328124,  1.        ]])

Good. Correlation coefficients are very small, because irrational voters (all independent) have utilities that are uncorrelated with those of other voters.

### This ended up being fodder for a unit test of the Irrational consideration

In [200]:
utilities = np.random.uniform(0., np.sqrt(12.), 50)
utilities = np.reshape(utilities, newshape=(5, 10))
utilities

array([[0.3090016 , 2.09904798, 0.82317008, 3.13087807, 1.61064933,
        2.01657254, 2.91759999, 1.55519431, 2.89235991, 2.97551052],
       [2.24721985, 2.65914812, 1.86329343, 2.21673783, 0.84797068,
        0.56396652, 0.44596168, 3.40639598, 0.65299382, 1.77379581],
       [0.58539738, 1.22377722, 3.06071522, 1.3184892 , 0.48842566,
        0.79741721, 2.61924207, 3.07438678, 1.22978133, 3.12792367],
       [2.85872826, 0.52028883, 1.38963001, 0.35922687, 0.16900879,
        1.34662276, 0.17156912, 1.73254542, 1.48895542, 1.27941031],
       [0.78623712, 0.66519608, 1.30919134, 2.73176796, 1.99823116,
        3.12736477, 3.38542127, 1.83569564, 3.25960743, 1.54455224]])

In [201]:
np.set_printoptions(threshold=100000)
print(str(utilities.transpose()))

[[0.3090016  2.24721985 0.58539738 2.85872826 0.78623712]
 [2.09904798 2.65914812 1.22377722 0.52028883 0.66519608]
 [0.82317008 1.86329343 3.06071522 1.38963001 1.30919134]
 [3.13087807 2.21673783 1.3184892  0.35922687 2.73176796]
 [1.61064933 0.84797068 0.48842566 0.16900879 1.99823116]
 [2.01657254 0.56396652 0.79741721 1.34662276 3.12736477]
 [2.91759999 0.44596168 2.61924207 0.17156912 3.38542127]
 [1.55519431 3.40639598 3.07438678 1.73254542 1.83569564]
 [2.89235991 0.65299382 1.22978133 1.48895542 3.25960743]
 [2.97551052 1.77379581 3.12792367 1.27941031 1.54455224]]


In [202]:
np.cov(utilities)

array([[ 0.94020538, -0.32073684,  0.16108757, -0.49611513,  0.62033183],
       [-0.32073684,  1.01169877,  0.31368278,  0.27153381, -0.70086438],
       [ 0.16108757,  0.31368278,  1.19057637, -0.00328159, -0.05275154],
       [-0.49611513,  0.27153381, -0.00328159,  0.71388613, -0.31396175],
       [ 0.62033183, -0.70086438, -0.05275154, -0.31396175,  1.02640331]])

In [220]:
utilities = np.random.uniform(0., np.sqrt(12.), 5000)
utilities = np.reshape(utilities, newshape=(5, 1000))
np.cov(utilities)

array([[ 0.99333244, -0.00444264,  0.01690256,  0.0234099 , -0.03744667],
       [-0.00444264,  1.01777285, -0.02395978, -0.02019903, -0.06810528],
       [ 0.01690256, -0.02395978,  0.99728841, -0.08037311, -0.03790731],
       [ 0.0234099 , -0.02019903, -0.08037311,  1.00854158,  0.02726133],
       [-0.03744667, -0.06810528, -0.03790731,  0.02726133,  0.99367376]])

In [26]:
config_issue = copy.deepcopy(config)
config_issue['candidates'] = 4
config_issue['voters'] = 100001
config_issue['considerations'] = yaml.safe_load(
'''
- Issues:
    issues:
    - halfcsep: 1.0
      halfvsep: 1.0
      sigma: 1.0
'''
)
result = do_run(config_issue, 5)
display(get_corr_coeffs(result, 0))
display(get_corr_coeffs(result, 1))
# display(get_corr_coeffs(result, 2))
# display(get_corr_coeffs(result, 3))

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.98468755,  1.        ,  0.        ,  0.        ],
       [ 0.74049005,  0.62866996,  1.        ,  0.        ],
       [-0.70489478, -0.79665994, -0.14975502,  1.        ]])

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.47824243,  1.        ,  0.        ,  0.        ],
       [ 0.98398031, -0.34422532,  1.        ,  0.        ],
       [-0.32372876,  0.97522388, -0.18019318,  1.        ]])

When there's a political axis, voters are much more significantly correlated with each other.

In [ ]:
df = run_experiment(cs(config, 'candidates', [2, 3, 4, 5, 6]), trials=20000)
df